# Sentiment Analysis Multi-Language

## 1. Initial Setup

In [ ]:
# Setup environment Windows
# python -m venv .venv_sentiment
# .venv_sentiment\Scripts\Activate.ps1  

# Linux
# sudo apt-get install python-pip
# $ pip install virtualenv
# virtualenv .venv_sentiment        # python3 -m venv .venv_sentiment
# source .venv_sentiment/bin/activate

# pip install -r requirements.txt
# python3 -m pip install tensorflow[and-cuda]
# pip install 'transformers[tf-cpu]'
# pip install 'transformers[flax]'


In [1]:
# Import libraries
import argparse
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
import os

# Import comments
path = "/home/ubuntu/thesis_GenAI/data/production_datasets/cleaned_datasets"
file_name = "comments_cleaned.csv"
file_path = os.path.join(path, file_name)
df_comments = pd.read_csv(file_path)
df_comments.head()

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


,stream_name,created_date,post_id,comment_id,comment_text,like_count,report_count,username,author_user_id,author_position,author_status
0,Water Break,2024-03-05 15:15:55,7612625,6866587,Congratulations and a great addition to the te...,0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
1,Water Break,2024-03-05 15:10:47,7612663,6866567,"Hello team, remember to have safety discussion...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
2,Water Break,2024-03-05 15:08:29,7613861,6866561,"@Larry_Broderick, both our moderation team and...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
3,Safety & Operations,2024-03-05 15:04:42,7613237,6866545,"@William_Pulliam, we appreciate you being prou...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active
4,Safety & Operations,2024-03-05 15:02:47,7613255,6866539,"We appreciate you supporting @Dennis_Cumbie, @...",0,0,FlaggerForce,b701ab9f-563a-4425-a389-aff803a8da58,NaN,active


In [2]:
parser = argparse.ArgumentParser("Sentiment Analysis")
parser.add_argument('comments_cleaned', help='dataframe with the text to analysis on pandas frame format')
parser.add_argument('comment_text', help='column in the file that contains the text')
args = parser.parse_args()

df_text = pd.read_csv(args.comments_cleaned)#, lineterminator='\n')
# model = 'finiteautomata/bertweet-base-sentiment-analysis'
model = 'cardiffnlp/twitter-xlm-roberta-base-sentiment'
sentiment = pipeline(task='sentiment-analysis', model=model)
counter = 0

# Define the total number of iterations
total_iterations = len(df_text)
# Create a progress bar
progress_bar = tqdm(total=total_iterations, position=0, leave=True)

for index, row in df_text.iterrows():
    try:
        df_text.loc[index, 'sentiment'] = str(sentiment(str(row[args.comment_text])[:279])[0]['label'])
        df_text.loc[index, 'sentiment_score'] = float(sentiment(str(row[args.comment_text])[:279])[0]['score'])
    except Exception as e:
        print(f'{str(row[args.comment_text])}', f'Error -> {e}')
    counter += 1
    progress_bar.update(1)  # Update the progress bar
    progress_bar.set_description(f'Processed: {counter}/{total_iterations}')

df_text.to_csv(f'sentiment_analysis_{args.file}', index=False)

usage: Sentiment Analysis [-h] comments_cleaned comment_text
Sentiment Analysis: error: the following arguments are required: comments_cleaned, comment_text


SystemExit: 2

/home/ubuntu/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
! pip install sentencepiece

In [8]:
import tensorflow as tf
from transformers import pipeline

# Suppress the deprecation warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Set up the sentiment analysis pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

# Perform sentiment analysis
result = sentiment_task("T'estimo!")
print(result)


ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [5]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("T'estimo!")

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [1]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

c:\Users\eduar\Documents\Master_Thesis\GenAI_Thesis_Beekeeper\thesis_GenAI\sentiment\.venv_sentiment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

text = "Good night 😊"
text = preprocess(text)
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)
scores = output[0][0].detach().numpy()
scores = softmax(scores)

ValueError: Couldn't instantiate the backend tokenizer from one of: 
(1) a `tokenizers` library serialization file, 
(2) a slow tokenizer instance to convert or 
(3) an equivalent slow tokenizer class to instantiate and convert. 
You need to have sentencepiece installed to convert a slow tokenizer to a fast one.

In [13]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-roberta-base-sentiment-latest"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("Covid cases are increasing fast!")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'label': 'negative', 'score': 0.7235770225524902}]

In [15]:
sentiment_task("you are an ugly bastard!")

[{'label': 'negative', 'score': 0.9404988288879395}]

In [10]:
from transformers import pipeline

distilled_student_sentiment_classifier = pipeline(
    model="lxyuan/distilbert-base-multilingual-cased-sentiments-student", 
    return_all_scores=True
)

# english
distilled_student_sentiment_classifier ("I love this movie and i would watch it again and again!")
# >> [[{'label': 'positive', 'score': 0.9731044769287109},
#   {'label': 'neutral', 'score': 0.016910076141357422},
#   {'label': 'negative', 'score': 0.009985478594899178}]]

# malay
distilled_student_sentiment_classifier("Saya suka filem ini dan saya akan menontonnya lagi dan lagi!")
# [[{'label': 'positive', 'score': 0.9760093688964844},
#   {'label': 'neutral', 'score': 0.01804516464471817},
#   {'label': 'negative', 'score': 0.005945465061813593}]]

# japanese
distilled_student_sentiment_classifier("私はこの映画が大好きで、何度も見ます！")
# >> [[{'label': 'positive', 'score': 0.9342429041862488},
#   {'label': 'neutral', 'score': 0.040193185210227966},
#   {'label': 'negative', 'score': 0.025563929229974747}]]


c:\Users\eduar\Documents\Master_Thesis\GenAI_Thesis_Beekeeper\thesis_GenAI\sentiment\.venv_sentiment\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\eduar\.cache\huggingface\hub\models--lxyuan--distilbert-base-multilingual-cased-sentiments-student. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\

[[{'label': 'positive', 'score': 0.9342427849769592},
  {'label': 'neutral', 'score': 0.04019320011138916},
  {'label': 'negative', 'score': 0.02556394413113594}]]